A with-as statement in Python is used in exception handling to make the code cleaner and much more readable. It simplifies the management of common resources like file streams. 

Observe the following code example related to file handling on how the use of with statement makes code cleaner.

## 1. No exception handling:
An exception during the file.write() call in the first implementation can prevent the file from closing properly which may introduce several bugs in the code, i.e. many changes in files do not go into effect until the file is properly  closed

In [2]:
file = open('066withasexample', 'w')
file.write('hello world !')
file.close()

## 2. Exception handling using try except/finally blocks
This is a proper way to take care of all exceptions but using the 'with' statement in the next example makes the code compact and much more readable.

In [3]:
file = open('file_path', 'w')
try:
    file.write('hello world')
finally:
    file.close()

## 3. using with statement

In [4]:
with open('file_path', 'w') as file:
    file.write('hello world !')

Notice that unlike the first two implementations, there is no need to call file.close() when using with statement. The with statement itself ensures proper acquisition and release of resources. Thus, with statement helps avoiding bugs and leaks by ensuring that a resource is properly released when the code using the resource is completely executed.

The with statement is popularly used with file streams, as shown above and with Locks, sockets, subprocesses and telnets etc.

# Supporting the “with” statement in user defined objects (Context Managers):
**Context managers** allow setup and cleanup actions to be taken for objects when their creation (constructor) is wrapped with a with statement. The behavior of the context manager is determined by two magic methods:

### \_\_enter\_\_(self)
Defines what the context manager should do at the beginning of the block created by the with statement. Note that the return value of \_\_enter\_\_ is bound to the target of the with statement, or the name after the as.

### \_\_exit\_\_(self, exception_type, exception_value, traceback)
Defines what the context manager should do after its block has been executed (or terminates). It can be used to handle exceptions, perform cleanup, or do something always done immediately after the action in the block. 

If the block executes successfully, exception_type, exception_value, and traceback will be None. Otherwise, you can choose to handle the exception or let the user handle it; if you want to handle it, make sure \_\_exit\_\_ returns True after all is said and done. If you don't want the exception to be handled by the context manager, just let it happen.

\_\_enter\_\_ and \_\_exit\_\_ can be useful for specific classes that have well-defined and common behavior for setup and cleanup. You can also use these methods to create generic context managers that wrap other objects.

Consider the following example for further clarification.

In [9]:
# a simple file writer object
class MessageWriter(object):
    def __init__(self, file_name):
        self.file_name = file_name

    def __enter__(self):
        self.file = open(self.file_name, 'w')
        return self.file

    def __exit__(self, exception_type, exception_value, traceback):
        self.file.close()


# using with statement with MessageWriter
with MessageWriter('my_file.txt') as xfile:
    xfile.write('hello world 2')

* In the code above, what follows the with keyword is the constructor of MessageWriter.
* As soon as the execution enters the context of the with statement a MessageWriter object is created and python then calls the \_\_enter\_\_() method. In this \_\_enter\_\_() method, initialize the resource you wish to use in the object.
* This  \_\_enter\_\_() method should always return a descriptor of the acquired resource.

## What are resource descriptors?
* These are the handles provided by the operating system to access the requested resources. In the precious code block, file is a descriptor of the file stream resource ie  file = open('hello.txt')
* In the MessageWriter example provided above, the \_\_enter\_\_() method creates a file descriptor and returns it. The name xfile here is used to refer to the file descriptor returned by the \_\_enter\_\_() method.
* The block of code which uses the acquired resource is placed inside the block of the with statement.
* As soon as the with block is executed, the \_\_exit\_\_() method is called. All the acquired resources are released in the \_\_exit\_\_() method. This is how we use the with statement with user defined objects.



# The contextlib module
[https://docs.python.org/3/library/contextlib.html]
> Note: Here, we require the knowledge of generators, decorators and yield.

A class based context manager as shown previously is not the only way to support the with statement in user defined objects. The contextlib module provides a few more abstractions built upon the basic context manager interface. Here is how we can rewrite the context manager for the MessageWriter object using the contextlib module

In [ ]:
from contextlib import contextmanager

class MessageWriter(object):
    def __init__(self, filename):
        self.file_name = filename

    @contextmanager
    def open_file(self):
        try:
            file = open(self.file_name, 'w')
            yield file
        finally:
            file.close()


# usage
message_writer = MessageWriter('hello.txt')
with message_writer.open_file() as my_file:
    my_file.write('hello world')

* In this code example, because of the yield statement in its definition, the function open_file() is a generator function.
* When this open_file() function is called, it creates a resource descriptor named file. This resource descriptor is then passed to the caller and is represented here by the variable my_file. After the code inside the with block is executed the program control returns back to the open_file() function. The open_file() function resumes its execution and executes the code following the yield statement. This part of code which appears after the yield statement releases the acquired resources. The @contextmanager here is a decorator.

The previous class-based implementation and this generator-based implementation of context managers is internally the same. While the later seems more readable, it requires the knowledge of generators, decorators and yield.